# DNA Language Model for Unsupervised Mutation Analysis

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 32
block_size = 256 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 100
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 10
n_embd = 64
n_head = 2
n_layer = 2
dropout = 0.3 # 0.2

## Data Loading and cleaning

In [ ]:
def prepare_fasta_for_gpt(input_file, output_file):
    with open(input_file, 'r') as f:
        lines = f.readlines()

    sequences = []
    current_seq = ""

    for line in lines:
        if line.startswith('>'):
            if current_seq: # If we have a sequence saved, add it to our list
                sequences.append(current_seq)
            current_seq = "" # Reset for the new sequence
        else:
            current_seq += line.strip().upper() # Clean and capitalize

    if current_seq: # Don't forget the last one
        sequences.append(current_seq)

    # Join with a separator that your tokenizer can recognize
    with open(output_file, 'w') as f:
        f.write("\n".join(sequences))

prepare_fasta_for_gpt(r"/content/sequence.fasta", 'dna_training_data.txt')
with open('dna_training_data.txt',"r",encoding="utf-8") as f:
    text = f.read()
print("length of dataset in characters:",len(text))

In [ ]:
torch.manual_seed(1337)
with open(r'/content/dna_training_data.txt','r', encoding='utf-8') as f:
    text = f.read().upper()


## Tokenization

In [ ]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string
print(f"Total characters: {len(text)}")
print(f"Unique characters: {set(text)}")

## Splitting into training and validation data

In [ ]:
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.95*len(data)) # first 95% will be train, rest val
train_data = data[:n]
val_data = data[n:]

## Creating Small batches

In [ ]:
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out


## Single Head

In [ ]:
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # input of size (batch, time-step, channels)
        # output of size (batch, time-step, head size)
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 # (B, T, hs) @ (B, hs, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,hs)
        out = wei @ v # (B, T, T) @ (B, T, hs) -> (B, T, hs)
        return out

## Multi-Head Attention

In [ ]:
class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel"""

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd,n_embd)
        self.dropout = nn.Dropout(dropout)
    def forward(self,x):
        out = torch.cat([h(x) for h in self.heads], dim = -1)
        out = self.proj(out)
        return out


class FeedForward(nn.Module):
    """ a simple linear layer followed by a non-linearity"""

    def __init__(self,n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd,4*n_embd),
            nn.ReLU(),
            nn.Linear(4*n_embd,n_embd),
            nn.Dropout(dropout),
        )

    def forward(self,x):
        return self.net(x)


## Transformer Block

In [ ]:
class Block(nn.Module):
    """Transformer block: communication followed by computation"""

    def __init__(self,n_embd,n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head,head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd) # using LayerNorm before forward pass in sa and feed forward
        self.ln2 = nn.LayerNorm(n_embd) # 0 mean , 1 deviation

    def forward(self,x):
        x = x +self.sa(self.ln1(x))
        x = x +self.ffwd(self.ln2(x))
        return x

## Model

In [ ]:
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd) # embed the vocab
        self.position_embedding_table = nn.Embedding(block_size,n_embd) # embed the block_size
        # self.sa_head = MultiHeadAttention(4,n_embd//4)
        # self.ffwd = FeedForward(n_embd)
        # self.blocks = nn.Sequential(
        #     Block(n_embd, n_head=4),
        #     Block(n_embd,n_head=4),
        #     Block(n_embd,n_head=4),
        #     nn.LayerNorm(n_embd),
        # )
        self.blocks = nn.Sequential(*[Block(n_embd,n_head = n_head) for _ in range(n_layer)])
        self.lm_head = nn.Linear(n_embd,vocab_size)

    def forward(self, idx, targets=None):
        B,T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (Batch_size, block_size, embed_C)
        pos_emb = self.position_embedding_table(torch.arange(T,device=device)) # (T,C)
        x = tok_emb + pos_emb #(B,T,C)
        # x = self.sa_head(x) # apply one head of self-attention. (B,T,C)
        # x = self.ffwd(x)
        x = self.blocks(x) #(B,T,C)
        logits = self.lm_head(x)# (Batch_size, block_size, vocab_size)
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:,-block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device)

## Training Loop to train the Model on the dataset

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()


## Instead of averaging the loss we take into consideration all the losses to get accurate results

In [ ]:
def per_token_loss(seq):
    model.eval()
    with torch.no_grad():
        idx = torch.tensor(encode(seq), dtype=torch.long, device=device).unsqueeze(0)
        x = idx[:, :-1]
        y = idx[:, 1:]

        logits, _ = model(x, y)   # logits shape: (B*T, vocab)
        T = y.numel()

        loss_per_token = F.cross_entropy(
            logits,
            y.view(-1),
            reduction="none"
        )

        return loss_per_token.view(T).cpu()

## Calculating the loss difference(Delta) between healthy and mutated DNA sequence

In [ ]:
def get_pathogenicity_score(healthy_seq, mutated_seq):
    lh = per_token_loss(healthy_seq)
    lm = per_token_loss(mutated_seq)

    delta_per_pos = lm - lh
    delta_mean = delta_per_pos.mean().item()
    delta_max = delta_per_pos.max().item()

    return {
        "delta_mean": delta_mean,
        "delta_max": delta_max,
        "delta_per_position": delta_per_pos
    }


healthy_dna = "ATGGTGCACCTGACTCCTGAGGAGAAG"
mutated_dna = "ATGGTGCACCTGACTCCTGTGGAGAAG"

result = get_pathogenicity_score(healthy_dna, mutated_dna)

print("Mean Δ:", result["delta_mean"])
print("Max Δ:", result["delta_max"])
print("Per-position Δ:", result["delta_per_position"])

## Plotting position in seq vs Loss Graph
### in the below graph we could see a peak in the 19th position as in the mutated sequence the change was in the 19th position there by also changing the loss of the next tokens

In [ ]:
import matplotlib.pyplot as plt

delta = result["delta_per_position"].numpy()

plt.figure()
plt.plot(delta)
plt.axhline(0)  # baseline
plt.xlabel("Position in sequence")
plt.ylabel("Δ Loss (Mutated − Healthy)")
plt.title("Per-position likelihood change due to mutation")
plt.show()

### In this example the mutated DNA is varied largely as compared to healthy DNA hence we could see the graph changing rapidely

In [ ]:
#Eample 2:
healthy_dna = "ATGGTGCACCTGACTCCTGAGGAGAAG"
mutated_dna = "AAAAAAAAAAAAAAAAAAAAAAAAAAA"

result = get_pathogenicity_score(healthy_dna, mutated_dna)

print("Mean Δ:", result["delta_mean"])
print("Max Δ:", result["delta_max"])
print("Per-position Δ:", result["delta_per_position"])
import matplotlib.pyplot as plt

delta = result["delta_per_position"].numpy()

plt.figure()
plt.plot(delta)
plt.axhline(0)  # baseline
plt.xlabel("Position in sequence")
plt.ylabel("Δ Loss (Mutated − Healthy)")
plt.title("Per-position likelihood change due to mutation")
plt.show()